<a href="https://colab.research.google.com/github/Clarice-Satiko-Aoto/b2b-analyis/blob/feat/VendasB2B_feat_eng_metas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install workalendar

# Bibliotecas

In [49]:
#importar bibliotecas
import pandas as pd

from workalendar.america import Brazil

# Crie um objeto de calendário brasileiro que exclui os feriados.
cal = Brazil()

#warnings
import warnings
warnings.filterwarnings("ignore")

# Carregar arquivos necessarios

In [25]:
#carregar arquivo

#df tratado
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/tratado.csv',
                 parse_dates=['data'])
#de para
dfDePara = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/de_para.csv')

#Meta 2022
meta22 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/Meta_2022.csv')

#Meta 2023
# mapeando o caminho
arquivo_excel = pd.ExcelFile('/content/drive/MyDrive/Colab Notebooks/vendas/data/Meta_2023.xlsx')
#lendo todas as tabelas
planilhas = arquivo_excel.parse(sheet_name=None,
                                header=2,
                                usecols=['Categoria Produto', 'Previsão Vendas', '1', '2', '3','4'])
#armazenando as planilhas
victor1_2023o = planilhas['1 - Victor Castro']
gabriel2_2023o = planilhas['2 - Gabriel Azevedo']

# Funções

In [26]:
def mudar_tipo_de_dados(dataframe, tipos_de_dados):
    """
    Muda o tipo de dados de colunas em um DataFrame.

    Parâmetros:
    - dataframe: DataFrame Pandas
    - tipos_de_dados: Lista de pares (nome da coluna, tipo de dados)
    """

    for coluna, tipo in tipos_de_dados:
        if coluna in dataframe.columns:
            dataframe[coluna] = dataframe[coluna].astype(tipo)

In [27]:
def dias_uteis_sem_feriado(df):
  """  dias uteis sem feriado no mes  """
  data_inicial = df.index.min()
  data_final = df.index.max()

  # Gere um índice de datas diárias.
  datas_diarias = pd.date_range(start=data_inicial, end=data_final, freq='D')

  # Filtrar as datas úteis, excluindo fins de semana e feriados brasileiros.
  dias_sem_feriado = [data for data in datas_diarias if data.weekday() < 5 and data not in cal.holidays()]
  return dias_sem_feriado

In [28]:
# Função para calcular a quantidade de dias úteis em um mês específico.

def calcular_dias_uteis_no_mes(row):
    """    calcular os dias uteis no mes    """
    ano, mes = row.name.year, row.name.month
    primeiro_dia = pd.Timestamp(year=ano, month=mes, day=1)
    ultimo_dia = pd.Timestamp(year=ano, month=mes, day=pd.Timestamp(year=ano, month=mes, day=1).days_in_month)

    datas_no_mes = pd.date_range(start=primeiro_dia, end=ultimo_dia)
    datas_uteis = [data for data in datas_no_mes if data.weekday() < 5]
    feriados = cal.holidays()
    datas_uteis = [data for data in datas_uteis if data not in feriados]
    return len(datas_uteis)

In [29]:
def transformar_meta_mensal_para_diaria(df):
  """Separando os dados de acordo com o nível gerente/categoria_id para que possa preencher
  com os valores da meta mensal no nível dias uteis"""

  meta_ano_gerente = []
  for ger in df['gerente_id'].unique():
    data = df[df['gerente_id']== ger]
    for cat in data['categoria_id'].unique():
      resample_ger_cat = data[data['categoria_id']==cat]
      df_diario = resample_ger_cat.reindex(dias_uteis_sem_feriado(resample_ger_cat))
      df_diario = df_diario.fillna(method='ffill')
      meta_ano_gerente.append(df_diario)
  # Concatene todos os DataFrames da lista em um único DataFrame.
  meta_concat = pd.concat(meta_ano_gerente)
  return meta_concat

In [30]:
#função para tratar os dados de 2023
#Padronizando a data
trimestre = {
    '1T': '2023-01-01',
    '2T': '2023-04-01',
    '3T': '2023-07-01',
    '4T': '2023-10-01'
}

def tratamento_dados_2023(df, id):
  #Adicionando a coluna com o gerente_id
  df['gerente_id'] = id

  #Extraindo apenas o id da categoria
  df['categoria_id'] = df['Categoria Produto'].str.extract('(\d+)')
  df.drop('Categoria Produto', axis=1, inplace=True)

  #calculando a meta trimestral
  df['1T'] = df['Previsão Vendas'] * df['1']
  df['2T'] = df['Previsão Vendas'] * df['2']
  df['3T'] = df['Previsão Vendas'] * df['3']
  df['4T'] = df['Previsão Vendas'] * df['4']

  #dropando as colunas que eu nao vou mais utilizar
  df.drop(columns=['Previsão Vendas', '1', '2', '3', '4'], axis=1, inplace=True)

  #utilizando a função metl para pivotar o dataframe
  df = pd.melt(df, id_vars=['gerente_id', 'categoria_id'], var_name='Trimestre', value_name='Valor')
  df.replace(trimestre, inplace=True)
  df['Trimestre'] = pd.to_datetime(df['Trimestre'])
  df.rename(columns={'Trimestre': 'data'}, inplace=True)
  # df.set_index('data', inplace=True)

  return df

In [31]:
def calcular_meta_diaria(df, col_meta):

  # Calcule os dias úteis para cada mês e adicione como uma nova coluna 'dias_uteis'.
  df['dias_uteis'] = df.apply(calcular_dias_uteis_no_mes, axis=1)

  # Distribua o valor de "$ Meta" igualmente pelos dias úteis.
  df['$_Meta_por_dia_util'] = round(df[col_meta] / df['dias_uteis'],2)
  df.reset_index(inplace=True)

  # filtrando as colunas que vamos utilizar e as datas do ano de 2022.
  meta_consolidado = df[['data','gerente_id', 'categoria_id', col_meta, '$_Meta_por_dia_util']]  #'dias_uteis',
  filtro_data = meta_consolidado['data'] >= '2022-01-01'
  meta_consolidado = meta_consolidado[filtro_data]
  return meta_consolidado


In [32]:
def calcular_meta_individual(df):
  """juntar a quantidade de funcionarios no datafrme
  das metas para calcular a meta diaria individual
  """
  meta_com_qde_func = df.merge(qde_func_por_gerente, on='gerente_id')
  meta_com_qde_func['meta_diaria_ind'] = round(meta_com_qde_func['$_Meta_por_dia_util'] / meta_com_qde_func['qde_funcionarios'],2)
  return meta_com_qde_func

In [33]:
def distribuir_meta_trimestral(df):
  """Transformar os dados mensais na granularidade diaria."""

  meta_tri = []
  for cat in df_metas_2023['categoria_id'].unique():
    filtro_cat = (df_metas_2023[df_metas_2023['categoria_id'] == cat])
    for ger in filtro_cat['gerente_id'].unique():
      filtro_ger = filtro_cat[filtro_cat['gerente_id']==ger]
      resample_cat_ger = filtro_ger.set_index('data').resample('D').ffill()
      df_diario = resample_cat_ger.reindex(dias_uteis_sem_feriado(resample_cat_ger))
      meta_tri.append(df_diario)
    #concatenando os dataframes
    meta_ger_23 = pd.concat(meta_tri)
    return meta_ger_23

# Dados auxiliares

## Mudar tipo de dado

In [34]:
tipos_de_dados = [('gerente_id', object), ('categoria_id', object), ('supervisor_id', object), ('funcionario_id', object)]

## Qde funcionarios por gerente

In [41]:
#Calculando q quantidade de funcionarios por gerente
qde_func_por_gerente = dfDePara['gerente_id'].value_counts().to_frame().reset_index().rename(columns={'gerente_id':'qde_funcionarios', 'index': 'gerente_id'})
qde_func_por_gerente

,gerente_id,qde_funcionarios
0,1,6
1,2,6


# Metas

## Meta 2022

In [35]:
#Padronizar a data
#Jan-22 deixei como dez/2021 para que possamos preencher as datas de janeiro com o método ffill
mes = {
    'Jan-22': '2021-12-31',
    'Fev-22': '2022-02-28',
    'Mar-22': '2022-03-31',
    'Abr-22': '2022-04-30',
    'Mai-22': '2022-05-31',
    'Jun-22': '2022-06-30',
    'Jul-22': '2022-07-31',
    'Ago-22': '2022-08-31',
    'Set-22': '2022-09-30',
    'Out-22': '2022-10-31',
    'Nov-22': '2022-11-30',
    'Dez-22': '2022-12-31',
}
meta22.replace(mes, inplace=True)
meta22['Mês'] = pd.to_datetime(meta22['Mês'])
meta22.rename(columns={'Mês': 'data',
                       '$ Meta': 'Valor'}, inplace=True)
meta22.set_index('data', inplace=True)

In [36]:
#mudar o tipo de dados
mudar_tipo_de_dados(meta22, tipos_de_dados)

In [37]:
#Transformar meta mensal para diária
meta22_concat = transformar_meta_mensal_para_diaria(meta22)

In [38]:
#Calcular a meta diaria(dias uteis)
meta22_consolidado = calcular_meta_diaria(meta22_concat, 'Valor')
meta22_consolidado.head()

,data,gerente_id,categoria_id,Valor,$_Meta_por_dia_util
1,2022-01-03,1,1,22586.0,1075.52
2,2022-01-04,1,1,22586.0,1075.52
3,2022-01-05,1,1,22586.0,1075.52
4,2022-01-06,1,1,22586.0,1075.52
5,2022-01-07,1,1,22586.0,1075.52


In [42]:
#Calcular meta individual diária
meta22_com_qde_func = calcular_meta_individual(meta22_consolidado)
meta22_com_qde_func.head()

,data,gerente_id,categoria_id,Valor,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind
0,2022-01-03,1,1,22586.0,1075.52,6,179.25
1,2022-01-04,1,1,22586.0,1075.52,6,179.25
2,2022-01-05,1,1,22586.0,1075.52,6,179.25
3,2022-01-06,1,1,22586.0,1075.52,6,179.25
4,2022-01-07,1,1,22586.0,1075.52,6,179.25


In [43]:
#Merge com o de para para termos as informaçoes do supervidor_id e vendedor_id
meta22_compilado = meta22_com_qde_func.merge(dfDePara, on='gerente_id')
meta22_compilado.head()

,data,gerente_id,categoria_id,Valor,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind,supervisor_id,vendedor_id
0,2022-01-03,1,1,22586.0,1075.52,6,179.25,1,12
1,2022-01-03,1,1,22586.0,1075.52,6,179.25,2,1
2,2022-01-03,1,1,22586.0,1075.52,6,179.25,2,4
3,2022-01-03,1,1,22586.0,1075.52,6,179.25,2,11
4,2022-01-03,1,1,22586.0,1075.52,6,179.25,3,6


## Meta 2023

### Gerente Victor

In [44]:
victor1_2023_1  = victor1_2023o.copy()

In [45]:
#Transformar os dados de 2023
df_victor0 = tratamento_dados_2023(victor1_2023_1,1)


In [46]:
df_victor = df_victor0.copy()
df_victor.head()

,gerente_id,categoria_id,data,Valor
0,1,1,2023-01-01,30000.0
1,1,2,2023-01-01,190000.0
2,1,3,2023-01-01,30000.0
3,1,4,2023-01-01,70000.0
4,1,1,2023-04-01,120000.0


In [47]:
#Como eu nao consegui preencher o dataframe com os dados de novembro e dezembro,
# resolvi fazer na mão mesmo, criando o dataframe e concatenando com o dataframe original

faltante = [
    ['2023-11-30', 1,'1',51000.0],
    ['2023-12-31', 1,'1',51000.0],
    ['2023-11-30', 1,'2',380000.0],
    ['2023-12-31', 1,'2',380000.0],
    ['2023-11-30', 1,'3',50000.0],
    ['2023-12-31', 1,'3',50000.0],
    ['2023-11-30', 1,'4',140000.0],
    ['2023-12-31', 1,'4',140000.0],
            ]
df_faltante = pd.DataFrame(faltante, columns=['data', 'gerente_id', 'categoria_id', 'Valor'])
# Transformando as colunas 'data' e 'categoria_id' em tipo datetime e object.
df_faltante['data'] = pd.to_datetime(df_faltante['data'])

#concatenando os dois dataframes
df_victor_c = pd.concat([df_victor, df_faltante], ignore_index=True)
df_victor_c.tail()

,gerente_id,categoria_id,data,Valor
19,1,2,2023-12-31,380000.0
20,1,3,2023-11-30,50000.0
21,1,3,2023-12-31,50000.0
22,1,4,2023-11-30,140000.0
23,1,4,2023-12-31,140000.0


In [48]:
#mudar o tipo de dados
mudar_tipo_de_dados(df_victor_c, tipos_de_dados)

### Gerente Gabriel

In [50]:
gabriel2_2023_2 = gabriel2_2023o.copy()

In [51]:
df_gabriel = tratamento_dados_2023(gabriel2_2023_2,2)
df_gabriel.head()

,gerente_id,categoria_id,data,Valor
0,2,1,2023-01-01,42500.0
1,2,2,2023-01-01,150000.0
2,2,3,2023-01-01,39000.0
3,2,4,2023-01-01,40000.0
4,2,1,2023-04-01,170000.0


In [52]:
#concatenando com os dados faltantes

#Como eu nao consegui preencher o dataframe com os dados de novembro e dezembro,
# resolvi fazer na mão mesmo, criando o dataframe e concatenando com o dataframe original

faltante = [
    ['2023-11-30', 2,'1',85000.0],
    ['2023-12-31', 2,'1',85000.0],
    ['2023-11-30', 2,'2',262500.0],
    ['2023-12-31', 2,'2',262500.0],
    ['2023-11-30', 2,'3',65000.0],
    ['2023-12-31', 2,'3',65000.0],
    ['2023-11-30', 2,'4',70000.0],
    ['2023-12-31', 2,'4',70000.0],
            ]
df_faltante = pd.DataFrame(faltante, columns=['data', 'gerente_id', 'categoria_id', 'Valor'])
# Transformando as colunas 'data' e 'categoria_id' em tipo datetime e object.
df_faltante['data'] = pd.to_datetime(df_faltante['data'])

#concatenando os dois dataframes
df_gabriel_c = pd.concat([df_gabriel, df_faltante], ignore_index=True)
df_gabriel_c.tail()

,gerente_id,categoria_id,data,Valor
19,2,2,2023-12-31,262500.0
20,2,3,2023-11-30,65000.0
21,2,3,2023-12-31,65000.0
22,2,4,2023-11-30,70000.0
23,2,4,2023-12-31,70000.0


In [53]:
#mudar o tipo de dados
mudar_tipo_de_dados(df_gabriel_c, tipos_de_dados)


Concatenar os dados dos dois gerentes

In [54]:
#concatenando os dados dos dois gerentes
df_metas_2023 = pd.concat([df_victor_c,df_gabriel_c ], ignore_index=True)
#adicionando a coluna trimestre para poder calcular os dias uteis
df_metas_2023['trimestre'] = df_metas_2023.data.dt.quarter
df_metas_2023.head()

,gerente_id,categoria_id,data,Valor,trimestre
0,1,1,2023-01-01,30000.0,1
1,1,2,2023-01-01,190000.0,1
2,1,3,2023-01-01,30000.0,1
3,1,4,2023-01-01,70000.0,1
4,1,1,2023-04-01,120000.0,2


In [55]:
feriados = cal.holidays()
df_resultado = pd.DataFrame()

for cat in df_metas_2023.categoria_id.unique():
  df_categoria = df_metas_2023.query("categoria_id == @cat")
  for ger in df_categoria.gerente_id.unique():
    df_gerente = df_categoria.query("gerente_id ==@ger")
    df_resampled = df_gerente.set_index('data').resample('D').ffill().reset_index()
    for tri in df_resampled.trimestre.unique():
      df_trimestre = df_resampled.query("trimestre == @tri")
      data_inicial = df_trimestre['data'].min()
      data_final = df_trimestre['data'].max()
      datas_trimestre = pd.date_range(start=data_inicial, end=data_final)
      datas_uteis = [data for data in datas_trimestre if data.weekday() < 5 ]
      datas_uteis = [data for data in datas_uteis if data not in feriados]
      quantidade_dias_uteis = len(datas_uteis)
      df_trimestre['dias_tri'] = quantidade_dias_uteis
      df_resultado = pd.concat([df_resultado, df_trimestre], ignore_index=True)

In [56]:
df_resultado['$_Meta_por_dia_util'] = df_resultado['Valor']/df_resultado['dias_tri']


In [58]:
df_resultado = df_resultado[['data', 'gerente_id', 'categoria_id', 'Valor', '$_Meta_por_dia_util']]
df_resultado.head()

,data,gerente_id,categoria_id,Valor,$_Meta_por_dia_util
0,2023-01-01,1,1,30000.0,461.538462
1,2023-01-02,1,1,30000.0,461.538462
2,2023-01-03,1,1,30000.0,461.538462
3,2023-01-04,1,1,30000.0,461.538462
4,2023-01-05,1,1,30000.0,461.538462


In [60]:
meta23_com_qde_func = calcular_meta_individual(df_resultado)
meta23_com_qde_func.head()

,data,gerente_id,categoria_id,Valor,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind
0,2023-01-01,1,1,30000.0,461.538462,6,76.92
1,2023-01-02,1,1,30000.0,461.538462,6,76.92
2,2023-01-03,1,1,30000.0,461.538462,6,76.92
3,2023-01-04,1,1,30000.0,461.538462,6,76.92
4,2023-01-05,1,1,30000.0,461.538462,6,76.92


In [61]:
df_resultado23 = meta23_com_qde_func[['data', 'gerente_id', 'categoria_id', 'Valor', '$_Meta_por_dia_util', 'meta_diaria_ind']]

In [62]:
df_resultado22 = meta22_com_qde_func[['data', 'gerente_id', 'categoria_id', 'Valor', '$_Meta_por_dia_util', 'meta_diaria_ind']]

In [63]:
metas_compiladas = pd.concat([df_resultado22, df_resultado23])

In [64]:
metas_compiladas

,data,gerente_id,categoria_id,Valor,$_Meta_por_dia_util,meta_diaria_ind
0,2022-01-03,1,1,22586.0,1075.520000,179.25
1,2022-01-04,1,1,22586.0,1075.520000,179.25
2,2022-01-05,1,1,22586.0,1075.520000,179.25
3,2022-01-06,1,1,22586.0,1075.520000,179.25
4,2022-01-07,1,1,22586.0,1075.520000,179.25
...,...,...,...,...,...,...
2915,2023-12-27,2,4,70000.0,1076.923077,179.49
2916,2023-12-28,2,4,70000.0,1076.923077,179.49
2917,2023-12-29,2,4,70000.0,1076.923077,179.49
2918,2023-12-30,2,4,70000.0,1076.923077,179.49


In [66]:
dfDePara

,supervisor_id,gerente_id,vendedor_id
0,1,1,12
1,2,1,1
2,2,1,4
3,2,1,11
4,3,1,6
5,3,1,7
6,4,2,2
7,4,2,5
8,4,2,8
9,5,2,3


Como não consegui fazer os merges sem perder os dados, vou separar os dados para fazer os merges e o concat

In [ ]:
# metas.data.min(), metas.data.max()

(Timestamp('2022-01-03 00:00:00'), Timestamp('2023-12-31 00:00:00'))

In [ ]:
# df_geral.data.min(), df_geral.data.max()

(Timestamp('2021-02-16 00:00:00'), Timestamp('2023-10-22 00:00:00'))

In [ ]:
# #juntando os dados em comum
# df_comum = df_geral.merge(metas, on=['data', 'supervisor_id', 'gerente_id', 'vendedor_id'])
# df_comum.data.min(), df_comum.data.max()

(Timestamp('2022-01-03 00:00:00'), Timestamp('2023-10-22 00:00:00'))

In [ ]:
# #1) pegar as informações de 2021
# df2021 = df_geral.query("data <2022")

In [ ]:
# # 2) pegar as informações a partir do dia 20/10/2023 do df metas
# metas3T = metas.query("data > '2023-10-22'")

In [ ]:
# # 3) concat dos tres df
# # em 2021, a empresa não havia estabelecido os valores das metas, logo, serão preenchidos com NaN.
# # temos datas futuras de 2023, logo, teremos valores NaN em datas superiores a 22/10/2023
# df_vendas_com_metas2023 = pd.concat([df2021, df_comum, metas3T])
# df_vendas_com_metas2023.head()

,vendas_id,data,nfe,cliente_id,nf_desconto,nf_valor_bruto,nf_valor_venda,itens_venda_id,produto_id,item_quantidade,...,categoria_id,descricao,tamanho,custoUnitario,categoria,margem_lucro_unit,categoria_id_x,categoria_id_y,Valor,$_Meta_por_dia_util
0,1.0,2021-02-16,SO43660,614.0,24.90,265.0,240.10,13.0,43.0,1.0,...,2,Paeonia,M,75.0,Manga Curta,37.0,NaN,NaN,NaN,NaN
2,993.0,2021-06-24,SO46324,259.0,62.80,1014.0,951.20,7650.0,43.0,1.0,...,2,Paeonia,M,75.0,Manga Curta,37.0,NaN,NaN,NaN,NaN
3,7.0,2021-02-17,SO43665,347.0,112.95,1711.0,1598.05,61.0,43.0,1.0,...,2,Paeonia,M,75.0,Manga Curta,37.0,NaN,NaN,NaN,NaN
5,12.0,2021-02-18,SO43670,266.0,26.35,602.0,575.65,112.0,43.0,1.0,...,2,Paeonia,M,75.0,Manga Curta,37.0,NaN,NaN,NaN,NaN
7,454.0,2021-04-26,SO44762,202.0,306.85,3776.0,3469.15,3627.0,43.0,1.0,...,2,Paeonia,M,75.0,Manga Curta,37.0,NaN,NaN,NaN,NaN


In [ ]:
# # salvando o arquivo tratado
# df_vendas_com_metas2023.to_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/tratado.csv', index=False)

verificar:


1)

